# Bio geolocation

Looks up the location of sequences in genbank and adds it to the FASTA file

In [108]:
import os
import re
import requests
import time
import json

In [9]:
# load the file
file = open("suillus.fas","r")
fas = file.read()
print(fas)

> Suillus MO # 243577
CCCTTCGTGCAGAAAGTCTATGAATGTTTTTACCATCATCGACTCGCGACTTCTAGGAGACGCGATTCTTTGAGACAAAAGTTATTACAACTTTCAGCAATGGATCTCTTGGCTCTCGCATCGATGAAGAACGCAGCGAATCGCGATATGTAATGTGAATTGCAGATCTACAGTGAATCATCGAATCTTTGAACGCACCTTGCGCTTATCGGTGTTCCGATGAGCATGCCTGTTTGAGCGTCATTAAATTCTCAACCCCTCTCGATTTGCTTCGAAGAGGGAGCTTGGATGGTGGAGGCTGCCGGAGACCTGTTTTTTCAGGACTCGGGCTCCTCTGAAATGTATCGGCTTGCGGTCGACTTTCGACtGTGCATGACAAGGCCTTTGGCGTGATAATGATCGCCGCTCGCCGAAGtGCACGAACGAATGGTCTCGCGCCTCTAATCAGTCGACGTCTTTTCGAAGGCGTCTTCCTCATCGACGTTTGACCTCAAATCAGGTAGGACTACCCGCTGAACTTAAGCATAtcata
>JQ711926.1 Suillus variegatus isolate FFP1068 18S ribosomal RNA gene, internal transcribed spacer 1, 5.8S ribosomal RNA gene, internal transcribed spacer 2, and 28S ribosomal RNA gene, region
TCCGGATCGGCTTCGGGGAGCCGGCGACGGCATCCTGTCGCTGAGAAGCTGATCAAACTTGGTCGTTTAGAGGAAGTAAA
AGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTAAAGAAATAATCTCGAGGGCCTTTGGAAAGGAGAGA
GGGTTGTAGCTGGCGTAAGCACGTGCACGCCCTCTTTCTCGACCTGGGTCCTTATGGGCGCGGGGCGACCCGCGTCTTCA
TAAGCCCCT

So we need to extract the country from genbank using this data. The format of the .fas file is as follows:

    > name
    sequence
    > name2
    sequence2

name is sometimes a Genbank accession ID, otherwise it's a hand written ID that we should ignore.

In [19]:
# separate the fas file into a list of sequences
sequences = fas.split('>')[1:]
sequences[1]

'JQ711926.1 Suillus variegatus isolate FFP1068 18S ribosomal RNA gene, internal transcribed spacer 1, 5.8S ribosomal RNA gene, internal transcribed spacer 2, and 28S ribosomal RNA gene, region\nTCCGGATCGGCTTCGGGGAGCCGGCGACGGCATCCTGTCGCTGAGAAGCTGATCAAACTTGGTCGTTTAGAGGAAGTAAA\nAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTAAAGAAATAATCTCGAGGGCCTTTGGAAAGGAGAGA\nGGGTTGTAGCTGGCGTAAGCACGTGCACGCCCTCTTTCTCGACCTGGGTCCTTATGGGCGCGGGGCGACCCGCGTCTTCA\nTAAGCCCCTTCGTGTAGAAAGTCTATGAATGTTTTTACCATCATCGACTCGCGACTTCTAGGAGACGCGATTCTTTGAGA\nCAAAAGTTATTACAACTTTCAGCAATGGATCTCTTGGCTCTCGCATCGATGAAGAACGCAGCGAATCGCGATATGTAATG\nTGAATTGCAGATCTACAGTGAATCATCGAATCTTTGAACGCACCTTGCGCTTATCGGTGTTCCGATGAGCATGCCTGTTT\nGAGCGTCATTAAATTCTCAACCCCTCTCGATTTGCTTCGAGAGGGAGCTTGGATAGTGGAGGCTGCCGGAGACCTGTTTT\nTTCAGGACTCGGGCTCCTCTGAAATGTATTGGCTTGCGGTCGACTTTCGACTGTGCATGACAAGGCCTTTGGCGTGATAA\nTGATCGCCGCTCGCCGAAGTGCACGAACGAATGGTCTCGTGCCTCTAATCAGTCGANGCCTTTTCGAAGGCGTCTTCCTC\nATTGACGTTTGACCTCAAATCAGGTAGGACTACCCGCTGAACTTAAGCATATCAATAAGCGGAGGAAA

In [48]:
# get the ID and name from the sequences
sequences_processed = []

for sequence in sequences:    
    try:
        # creating a local dict
        seq = {}

        # assession
        assession_regex = re.search(r'([A-Z]+\d+)', sequence)
        seq['assession'] = assession_regex.group()
        
        # name
        split_sequence = sequence.split(' ')
        seq['name'] = "{} {}".format(split_sequence[1], split_sequence[2])
        
        # add to dict
        sequences_processed.append(seq)
    except:
        continue

In [49]:
sequences_processed

[{'assession': 'JQ711926', 'name': 'Suillus variegatus'},
 {'assession': 'FJ475559', 'name': 'Uncultured Suillaceae'},
 {'assession': 'FN565360', 'name': 'Uncultured Suillus'},
 {'assession': 'FN565362', 'name': 'Uncultured Suillus'},
 {'assession': 'FN565363', 'name': 'Uncultured Suillus'},
 {'assession': 'JQ753773', 'name': 'Suillus variegatus'},
 {'assession': 'JQ711888', 'name': 'Suillus sp.'},
 {'assession': 'JQ711867', 'name': 'Suillus variegatus'},
 {'assession': 'JQ177144', 'name': 'Uncultured Suillus'},
 {'assession': 'FN565367', 'name': 'Uncultured Suillus'},
 {'assession': 'FN565370', 'name': 'Uncultured Suillus'},
 {'assession': 'FJ660476', 'name': 'Uncultured ectomycorrhizal'},
 {'assession': 'AY898622', 'name': 'Suillus variegatus'},
 {'assession': 'AM086444', 'name': 'Suillus variegatus'},
 {'assession': 'AJ272418', 'name': 'Suillus variegatus'},
 {'assession': 'AJ272405', 'name': 'Suillus sp.'},
 {'assession': 'KM248962', 'name': 'Suillus tomentosus'},
 {'assession': 'F

ok so now we must do the request to the NCBI database to get the geolocation

In [106]:
url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id={}&rettype=gb"
sequences_with_geolcation = []

for sequence in sequences_processed:
    # local dict
    seq = {}
    seq['name'] = sequence['name']
    
    # formatting url
    seq['assession'] = str(sequence['assession'])
    formatted_url = url.format(seq['assession'])

    # getting the data and splitting it into lines
    resp = requests.get(formatted_url)
    resp_text = resp.text
    resp_lines = resp_text.split('\n')

    # ugly loop that checks if there is a country or lat_lon
    # variable and saves the output if at least one of them
    # is true
    for line in resp_lines:
        # regex search
        regex = re.search(r'(/country|/lat_lon)', line)
        try:
            # returns none if no matches found
            if regex.group():
                if "/country=" in line.split('"')[0]:
                    seq['country'] = line.split('"')[1]
                if "/lat_lon=" in line.split('"')[0]:
                    seq['lat_lon'] = line.split('"')[1]
        except:
            continue

    # append to list
    sequences_with_geolcation.append(seq)
    
    # don't kill the server and sleep for 200ms
    time.sleep(0.2)

In [107]:
sequences_with_geolcation

[{'assession': 'JQ711926',
  'country': 'Canada: BC',
  'lat_lon': '54.82 N 126.80 W',
  'name': 'Suillus variegatus'},
 {'assession': 'FJ475559',
  'country': 'Sweden',
  'name': 'Uncultured Suillaceae'},
 {'assession': 'FN565360',
  'country': 'United Kingdom:Scotland, Grantown on Spey -',
  'lat_lon': '57.19 N 3.37 W',
  'name': 'Uncultured Suillus'},
 {'assession': 'FN565362',
  'country': 'United Kingdom:Scotland, Grantown on Spey -',
  'lat_lon': '57.19 N 3.37 W',
  'name': 'Uncultured Suillus'},
 {'assession': 'FN565363',
  'country': 'United Kingdom:Scotland, Grantown on Spey -',
  'lat_lon': '57.19 N 3.37 W',
  'name': 'Uncultured Suillus'},
 {'assession': 'JQ753773', 'name': 'Suillus variegatus'},
 {'assession': 'JQ711888',
  'country': 'Canada: BC',
  'lat_lon': '52.83 N 123.73 W',
  'name': 'Suillus sp.'},
 {'assession': 'JQ711867',
  'country': 'Canada: BC',
  'lat_lon': '52.83 N 123.73 W',
  'name': 'Suillus variegatus'},
 {'assession': 'JQ177144',
  'country': 'United Ki

Cool so it worked! Let's save it :D

In [110]:
with open('suillus_processed.json', 'w') as fp:
    json.dump(sequences_with_geolcation, fp)

Ok so actually this needs to be formatted into the `.fas` file that was given..

In [177]:
sequences_formatted_as_fas = []

for s in range(len(sequences)):
    # getting the right sequence data from two different lists
    original = sequences[s+1]
    processed = sequences_with_geolcation[s]
    
    # finding the data we need
    actual_sequence_dirty = original.split('\n',1)[1:]
    actual_sequence = ''.join(actual_sequence_dirty) # this makes a list into a string
    name = processed['name']
    assession = processed['assession']
    country = processed.get('country')
    #print(">",assession,name,country,"\n",actual_sequence)
    
    # adding it quickly to a list
    sequence_formatted = "> ",assession, " ", name, " ",country,"\n",actual_sequence
    sequences_formatted_as_fas.append(sequence_formatted)
    
    # will figure this out another day
    #try:
        #lines = sequence.split('\n',1)[1:]
        #assession_regex = re.search(r'([A-Z]+\d+)', sequence)
        #assession = assession_regex.group()
        #full = sequences_with_geolcation[sequence]
        #if len(lines) < 1:
        #    continue
            #print(len(lines))
            #words = sequence.split(' ')
            #print(words)
            #for string in words:
            #    print(len(string))
        #print(full)
    #except:
        #continue


IndexError: list index out of range

even though the above command errored out, it still worked

In [175]:
output = open('suillus_processed.fas', 'w')

for seq in sequences_formatted_as_fas:
    try:
        s = ''.join(seq)
        output.write(s)
    except:
        pass

output.close()